In [0]:
import os
import pandas as pd
import numpy as np
import timeit
import ujson
#Usamos ujson que parece que es el que hace la importación más rápida. Añadir la documentación donde se comenta.
#from google.colab import drive  #Lo pongo más abajo, porque si no me da error (por qué??!!)

In [2]:
!ls -lh

total 3.2G
-rw-r--r-- 1 root root 2.5K Oct 28 10:31 adc.json
-rw-r--r-- 1 root root 2.7M Oct 28 16:36 Cast_Crew_ids.csv
drwx------ 3 root root 4.0K Oct 28 10:34 drive
-rw-r--r-- 1 root root 1.5M Oct 28 16:36 IMDB_5000_movie.csv
-rw-r--r-- 1 root root 124K Oct 28 16:36 Movies_ids.csv
-rw-r--r-- 1 root root 1.5G Oct 28 17:54 movies.zip
-rw-r--r-- 1 root root 1.8G Oct 28 16:36 people.zip
drwxr-xr-x 2 root root 4.0K Oct 25 17:24 sample_data
drwxr-xr-x 4 root root 4.0K Oct 28 11:33 themoviedb_data


##IMPORTACIÓN DESDE GOOGLE DRIVE

In [0]:
#Esto sólo funciona en Google Colab porque tiene una librería de Google Colab.
#Más documentación en:
#https://colab.research.google.com/notebooks/io.ipynb
#en el apartado PyDrive.
#PyDrive documentación:
#https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz

In [0]:
#Estos ficheros están en mi google drive.

#Vínculo a la carpeta comprimida con todas las pelis en mi Google Drive:
#https://drive.google.com/open?id=1-jrX_p1VfgMceGIj7SK_eIef8syIGsdL

#movies
file_id = '1-jrX_p1VfgMceGIj7SK_eIef8syIGsdL'

downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("movies.zip")

#trial
#file_id = '1ivGsdMnbPyHuxXPoZAnQVnjKp82-hq9o'

#downloaded = drive.CreateFile({'id': file_id})
#downloaded.GetContentFile("trial.zip")

In [5]:
!ls -lh

total 3.2G
-rw-r--r-- 1 root root 2.5K Oct 28 10:31 adc.json
-rw-r--r-- 1 root root 2.7M Oct 28 16:36 Cast_Crew_ids.csv
drwx------ 3 root root 4.0K Oct 28 10:34 drive
-rw-r--r-- 1 root root 1.5M Oct 28 16:36 IMDB_5000_movie.csv
-rw-r--r-- 1 root root 124K Oct 28 16:36 Movies_ids.csv
-rw-r--r-- 1 root root 1.5G Oct 28 17:54 movies.zip
-rw-r--r-- 1 root root 1.8G Oct 28 16:36 people.zip
drwxr-xr-x 2 root root 4.0K Oct 25 17:24 sample_data
drwxr-xr-x 4 root root 4.0K Oct 28 11:33 themoviedb_data


In [6]:
#Da un error pero no tiene consecuencias

#!mkdir themoviedb_data
#!cd themoviedb_data && unzip -q ../trial.zip
#!ls -hl themoviedb_data

!mkdir themoviedb_data
!cd themoviedb_data && unzip -q ../movies.zip
!ls -hl themoviedb_data

warning [../movies.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [../movies.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #409792).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)
total 12M
drwxr-xr-x 2 root root 12M Oct 28 10:33 movies


In [7]:
!ls -hl ./themoviedb_data/movies | head -10

total 4.3G
-rw-r--r-- 1 root root 7.4K Aug 26 18:10 100009.json
-rw-r--r-- 1 root root  22K Aug 25 12:44 10000.json
-rw-r--r-- 1 root root  24K Aug 26 18:10 100010.json
-rw-r--r-- 1 root root  18K Aug 26 18:10 100011.json
-rw-r--r-- 1 root root  16K Aug 26 18:10 100013.json
-rw-r--r-- 1 root root 6.8K Aug 26 18:10 100016.json
-rw-r--r-- 1 root root  19K Aug 26 18:10 100017.json
-rw-r--r-- 1 root root  32K Aug 25 12:44 10001.json
-rw-r--r-- 1 root root  16K Aug 26 18:10 100022.json


In [6]:
!ls -1 themoviedb_data/movies | wc -l

409791


##ACCESO A MI DRIVE

In [9]:
#Esto me va a permitir tener acceso a las carpetas de mi drive desde Colab:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
!ls "/content/drive/My Drive/" -lh

total 3.4G
drwx------ 2 root root 4.0K Jul 14 07:08  20180714_Deep_Learning
-rw------- 1 root root  76K Oct 22 18:13  6885.json
drwx------ 2 root root 4.0K Jun 26  2016 'avast! Files (XT1068, UWbYzWPVmAmwlrlD)'
drwx------ 2 root root 4.0K Jul 18 18:38 'Colab Notebooks'
-rw------- 1 root root  44K Feb  4  2018  contacts.vcf
-rw------- 1 root root 170M Oct 22 12:05  dfMoviesCast.csv
-rw------- 1 root root    1 Apr  9  2018  facturaIVA.gsheet
-rw------- 1 root root 1.5M Oct 24 17:34  IMDB_5000_movie.csv
-rw------- 1 root root    1 Oct 28 09:54 'Mis cuentas.gsheet'
-rw------- 1 root root 1.5G Oct 12 15:09  movies.zip
-rw------- 1 root root    1 Oct 20 08:40 'Nuestras Cuentas.gsheet'
-rw------- 1 root root 1.8G Oct 27 13:06  people.zip
-rw------- 1 root root    1 Jul 24 18:17 'Propuesta de una idea.gslides'
drwx------ 2 root root 4.0K Apr  7  2015  Screencastify
-rw------- 1 root root    1 Sep 30  2017  Series.gdoc
-rw------- 1 root root    1 Jan  2  2016 'Trabajos Iñigo.gsheet'
-rw-------

## ESTE BLOQUE CONTIENE EL CÓDIGO QUE HE DE EJECUTAR PARA ESTRUCTURAR MIS DATOS:

EL DETALLE DE CÓMO HE LLEGADO A ESTE CÓDIGO ESTÁ EN EL NOTEBOOK 03_TIDYING_MOVIES_DATA.IPYNB

In [4]:
#Lo importo todo. Tengo que reducir el número de campos para que me quepa en memoria.
#Excluyo: backdrop_path, external_ids, homepage, images_backdrops, images_posters, poster_path, similar_movies_page/results/total_pages/total_results,
#video, recommendations_page/results/total_pages/total_results
#Recommendations: películas recomendadas para una película. No es lo mismo que similar_movies.
#Para hacer esto debería hacer una importación inicial de todos los campos y luego, una vez decidido, importo todo.

#Usamos un generator, que es más eficiente en cuanto a memoria. "yield" va asociado al generator.
#Poner la nota: importo todo, aunque sé que lo voy a usar, por si alguien puede querer usarlo.
#Desanido el primer nivel de diccionarios. Para belongs_to_collection, que a veces es nulo en lugar de un diccionario, tomo dos keys.
#Me queda tratar las listas que contienen diccionarios. 
#¿Qué pasa con UTF8? !!!!!!!!!!!!!!!!!!!!!!!!!!
#No vale de nada cambiar el orden!!! Así que mantengo el alfabético del csv
#Por mi código anterior ya sé que adult es un booleano y que no tiene nulos. Así que puedo filtrarlo por adult = False. Voy a excluir estas
#pelis de mi análisis.

#Sobre mi notebook 03 he incluido la variable imdb_id, que necesito para el cruce posterior que hago.

path = "./themoviedb_data/movies"
dfMovies = pd.DataFrame()

def moviesIterator(path):
    intI = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            intI += 1
            if f.endswith('.json'):
                fp = os.path.join(root,f)
                with open(fp) as o:
                    data = ujson.load(o)
                
                if intI % 10000 == 0:
                    print(intI)
                    
                result = {"titles": data["alternative_titles"]["titles"], 
                       "budget": data["budget"], "credits_cast": data["credits"]["cast"], "credits_crew": data["credits"]["crew"],
                       "genres": data["genres"], "id": data["id"], "imdb_id": data["imdb_id"],
                       "keywords": data["keywords"]["keywords"], "original_language": data["original_language"],
                       "original_title": data["original_title"], "overview": data["overview"],
                       "popularity": data["popularity"], 
                       "production_companies": data["production_companies"], "production_countries": data["production_countries"],
                       "release_date": data["release_date"], "release_dates": data["release_dates"]["results"],
                       "revenue": data["revenue"], 
                       "reviews_page": data["reviews"]["page"],
                       "reviews_results": data["reviews"]["results"],
                       "reviews_total_pages": data["reviews"]["total_pages"],
                       "reviews_total_results": data["reviews"]["total_results"],
                       "runtime": data["runtime"],
                       "spoken_languages": data["spoken_languages"], "status": data["status"],
                       "tagline": data["tagline"], "title": data["title"], 
                       "vote_average": data["vote_average"], "vote_count": data["vote_count"]}
                    
                if data["belongs_to_collection"] is None:
                    result["belongs_to_collection_id"] = None
                    result["belongs_to_collection_name"] = None
                else:
                    result["belongs_to_collection_id"] = data["belongs_to_collection"]["id"]
                    result["belongs_to_collection_name"] = data["belongs_to_collection"]["name"]
                    
                if data["adult"] == False:
                    yield result
                      
tic=timeit.default_timer()
dfMovies = pd.DataFrame(moviesIterator(path))
#dfMovies.set_index('id', inplace=True)
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000


281.6626627250007

In [6]:
dfMovies['id'].tail()

393234    454504
393235    344624
393236    492673
393237    127030
393238    422674
Name: id, dtype: int64

In [0]:
#Esto lo hago para que mi código de crear tablas intermedias sea más rápido.
dfMovies.index = dfMovies['id']

In [8]:
dfMovies['id'].tail()

id
454504    454504
344624    344624
492673    492673
127030    127030
422674    422674
Name: id, dtype: int64

In [0]:
#columnName: columna de dfMovies a tratar. Sus elemeentos son listas de diccionarios.
#args: lista variable de claves con las que me voy a quedar de los diccionarios.
#Si un elemento es una lista vacía, no hay problema.
def columnIteratorMovies(columnName, *args):
    for id in dfMovies.index:
        for dict in dfMovies[dfMovies.index == id][columnName].iloc[0]:
            outputDict = {'movie_id': id}
            for key in args:
                outputDict[key] = dict.get(key,None)
            yield outputDict

In [0]:
#Creo mi dataframe con la info de la columna correspondiente, compruebo que tiene el nº filas que ha de tener y, si es así, elimino la columna
#correspondiente en dfMovies. Haré todo esto con la siguiente función:
def columnProcessesMovies (columnName, *args):
    #I need to declare dfMovies as global as I will change it inside this function.
    global dfMovies
    
    dfOutput = pd.DataFrame()
    dfOutput = pd.DataFrame(columnIteratorMovies(columnName, *args))
    
    #Compruebo que tiene el nº de registros que ha de tener.
    dfCheck = pd.DataFrame(dfMovies[columnName].map(len).value_counts())
    dfCheck['calculation']=dfCheck[columnName]*dfCheck.index
    
    if dfOutput.shape[0] == dfCheck['calculation'].sum():
        dfMovies = dfMovies.drop(columnName, axis=1)
        return dfOutput
    else:
        return "Error in process"

In [0]:
def columnIteratorReleaseDates(*args):
    for id in dfMoviesReleaseDates1.index:
        for dict in dfMoviesReleaseDates1[dfMoviesReleaseDates1.index == id]['release_dates'].iloc[0]:
            outputDict = {'movie_id': dfMoviesReleaseDates1[dfMoviesReleaseDates1.index == id]['movie_id'].iloc[0],
                         'iso_3166_1': dfMoviesReleaseDates1[dfMoviesReleaseDates1.index == id]['iso_3166_1'].iloc[0]}
            for key in args:
                outputDict[key] = dict[key]
            yield outputDict

In [0]:
#Generación de todas las tablas de una tacada: 
#This module took about 70min to be executed in Google Colab.
tic=timeit.default_timer()

dfMoviesCast = columnProcessesMovies ('credits_cast', 'cast_id', 'character', 'gender', 'id', 'name', 'order')
print("dfMoviesCast generated")
dfMoviesCrew = columnProcessesMovies ('credits_crew', 'department', 'gender', 'id', 'job', 'name')
print("dfMoviesCrew generated")
dfMoviesGenres = columnProcessesMovies ('genres', 'id', 'name')
print("dfMoviesGenres generated")
dfMoviesKeywords = columnProcessesMovies ('keywords', 'id', 'name')
print("dfMoviesKeywords generated")
dfMoviesProductionComp = columnProcessesMovies ('production_companies', 'id', 'name', 'origin_country')
print("dfMoviesProductionComp generated")
dfMoviesProductionCountries = columnProcessesMovies ('production_countries', 'iso_3166_1', 'name')
print("dfMoviesProductionCountries generated")
dfMoviesReviewsResults = columnProcessesMovies ('reviews_results', 'author', 'content', 'id')
print("dfMoviesReviewsResults generated")
dfMoviesSpokenLang = columnProcessesMovies ('spoken_languages', 'iso_639_1', 'name')
print("dfMoviesSpokenLang generated")
dfMoviesReleaseDates1 = columnProcessesMovies ('release_dates', 'iso_3166_1', 'release_dates')
print("dfMoviesReleaseDates1 generated")
dfMoviesReleaseDates = pd.DataFrame(columnIteratorReleaseDates('certification', 'iso_639_1', 'release_date', 'type'))
print("dfMoviesReleaseDates generated")

#Libero la memoria (realmento no lo hace...)
dfMoviesReleaseDates1 = pd.DataFrame()
%xdel dfMoviesReleaseDates1

toc=timeit.default_timer()
toc - tic #elapsed time in seconds

In [0]:
dfMovies.columns

##EXPLORATORY DATA ANALYSIS FOR MOVIES

In [0]:
dfMovies.columns

In [0]:
dfMovies.sample(3)

####Intento hacerlo con altair pero me encuentro con que hay un límite para 5mil registros.¿Quizá puedo primero agregar antes de representar?

In [0]:
#Intento hacerlo con altair pero me encuentro con que hay un límite para 5mil registros.
#¿Quizá puedo primero agregar antes de representar?

In [0]:
!pip install -U altair vega_datasets notebook vega

In [0]:
import altair as alt
from vega_datasets import data

In [0]:
#Si estoy en un notebook normal, ejecutaría:
#alt.renderers.enable("notebook")

#Pero en colab, ejecuto:
alt.renderers.enable("colab")

In [0]:
#Y aquí tengo el error.
alt.Chart(dfMovies).mark_bar().encode(
   #Como quiero hacer una modificación de x, tengo que incluir alt.X. Si fuese de y, sería alt.Y 
   x=alt.X("status", bin=True),
   y="count()" 
)

####Así que de momento lo hago con matplotlib o seaborn

In [0]:
#Si uso el código de Toni:
import matplotlib.pyplot as plt
%matplotlib inline  
#Para que nos haga los plot en la misma celda y que no salga una nueva pantalla
plt.style.use('ggplot')   
#Esto es para el estilo, pero da un poco igual

In [0]:
#Si uso seaborn, de los cursos de DataCamp
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline  
sns.set()
#Esto es para el estilo, pero da un poco igual

####EXPLORACIÓN INICIAL:

In [0]:
#Veo qué columnas tienen algún nulo:
dfMovies.isnull().any()

In [0]:
dfMovies['status'].value_counts()

In [0]:
#No entiendo cómo tienen ingresos las películas que no han sido "Released"
#Como revenue no tiene nulos, puedo contarlo para conocer el nº de registros.
dfMovies.groupby('status')['revenue'].agg(['sum','count'])

In [0]:
#Debería quedarme en mi modelo sólo con las "Released". Podría después aplicarlo al resto.

In [0]:
#Me interesa ver de las pelis de antes cuántas tienen revenue!=0 (para ver si son solo unas pocas pelis las que producen ingreso cuando status!=Released)
dfMovies[dfMovies['revenue']!=0].groupby('status')['revenue'].agg(['sum','count'])

In [0]:
dfMovies.shape[0]

In [0]:
dfMovies[dfMovies['revenue']!=0].shape[0]

In [0]:
dfMovies[dfMovies['revenue']>50].shape[0]

In [0]:
#En principio, registros que tendría para mi modelo. ¿Excluiría las pelis con ingresos >0 y <50? Habría que echar un vistazo.
#¿Excluiría las pelis de este año? Porque quizás los datos de ingresos no se han estabilizado.
dfMovies[dfMovies['revenue']>50][dfMovies['status']=='Released'].shape[0]

In [0]:
#¿De cuántas pelis tengo el presupuesto?
dfMovies[dfMovies['budget']>50][dfMovies['status']=='Released'].shape[0]

In [0]:
#¿De cuántas pelis coincide que tengo presupuesto e ingresos?
dfMovies[dfMovies['revenue']>50][dfMovies['budget']>50][dfMovies['status']=='Released'].shape[0]

In [0]:
#Relajando el criterio de >50:
dfMovies[(dfMovies['revenue']>0) & (dfMovies['budget']>0) & (dfMovies['status']=='Released')].shape[0]

In [0]:
dfMovies[dfMovies['title']=='Avatar'][['release_date','credits_cast']]

####VOY A TRAERME OTRO DATASET DE 5000 PELIS DE IMDB PARA MEJORAR LA INFO DE BUDGET Y REVENUE.

In [0]:
#¿Con qué criterio estará la info en este dataset?!!!!

In [0]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#5000 movies from IMDB.
#Tuve que volver a ejecutar el código de las librerías de Google, porque si no ésta da error. Ejecuntando la anterior, veo que funciona.
#¿Puedo eliminar alguna línea del código de la celda anterior esta vez?

file_id = '1ebj6EkGylXTGldfGfJO_AHEAncNf4zKA'

downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("IMDB_5000_movie.csv")


In [15]:
!ls -lh

total 3.2G
-rw-r--r-- 1 root root 2.5K Oct 28 10:31 adc.json
-rw-r--r-- 1 root root 2.7M Oct 28 16:36 Cast_Crew_ids.csv
drwx------ 3 root root 4.0K Oct 28 10:34 drive
-rw-r--r-- 1 root root 1.5M Oct 28 20:23 IMDB_5000_movie.csv
-rw-r--r-- 1 root root 124K Oct 28 16:36 Movies_ids.csv
-rw-r--r-- 1 root root 1.5G Oct 28 17:54 movies.zip
-rw-r--r-- 1 root root 1.8G Oct 28 16:36 people.zip
drwxr-xr-x 2 root root 4.0K Oct 25 17:24 sample_data
drwxr-xr-x 4 root root 4.0K Oct 28 11:33 themoviedb_data


In [107]:
!head IMDB_5000_movie.csv 

color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
Color,James Cameron,723,178,0,855,Joel David Moore,1000,760505847,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar ,886204,4834,Wes Studi,0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1,3054,English,USA,PG-13,237000000,2009,936,7.9,1.78,33000
Color,Gore Verbinski,302,169,563,1000,Orlando Bloom,40000,309404152,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End ,471220,48350,Jack Davenport,0,goddess|marriage ceremony|marriage proposal|pirate|singapore,http://www.imdb.com/title/tt0449088/?ref_=fn_tt_tt_1,

In [16]:
my_file_path='./IMDB_5000_movie.csv'

#Reading file and saving in df
dfMovies5000 = pd.read_csv(my_file_path)
dfMovies5000.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [17]:
#Miro el final a ver si hay algo raro:
dfMovies5000.tail()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
5038,Color,Scott Smith,1.0,87.0,2.0,318.0,Daphne Zuniga,637.0,NaN,Comedy|Drama,...,6.0,English,Canada,NaN,NaN,2013.0,470.0,7.7,NaN,84
5039,Color,NaN,43.0,43.0,NaN,319.0,Valorie Curry,841.0,NaN,Crime|Drama|Mystery|Thriller,...,359.0,English,USA,TV-14,NaN,NaN,593.0,7.5,16.00,32000
5040,Color,Benjamin Roberds,13.0,76.0,0.0,0.0,Maxwell Moody,0.0,NaN,Drama|Horror|Thriller,...,3.0,English,USA,NaN,1400.0,2013.0,0.0,6.3,NaN,16
5041,Color,Daniel Hsia,14.0,100.0,0.0,489.0,Daniel Henney,946.0,10443.0,Comedy|Drama|Romance,...,9.0,English,USA,PG-13,NaN,2012.0,719.0,6.3,2.35,660
5042,Color,Jon Gunn,43.0,90.0,16.0,16.0,Brian Herzlinger,86.0,85222.0,Documentary,...,84.0,English,USA,PG,1100.0,2004.0,23.0,6.6,1.85,456


In [18]:
dfMovies5000.shape

(5043, 28)

In [19]:
dfMovies5000.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [20]:
#De aquí tengo que extraer la clave de imdb para relacionarla con dfMovies.
dfMovies5000['movie_imdb_link'].sample(5)

3970    http://www.imdb.com/title/tt0031381/?ref_=fn_t...
598     http://www.imdb.com/title/tt0277434/?ref_=fn_t...
3185    http://www.imdb.com/title/tt0279286/?ref_=fn_t...
1300    http://www.imdb.com/title/tt0375173/?ref_=fn_t...
391     http://www.imdb.com/title/tt0352248/?ref_=fn_t...
Name: movie_imdb_link, dtype: object

In [21]:
#Voy a ver primero cuántos nulos tengo por campo. ¿Tiene nulos movie_imdb_link?
dfMovies5000.isnull().any()
#Hay muchas columnas con nulos, pero movie_imdb_link no tiene ninguno.

color                         True
director_name                 True
num_critic_for_reviews        True
duration                      True
director_facebook_likes       True
actor_3_facebook_likes        True
actor_2_name                  True
actor_1_facebook_likes        True
gross                         True
genres                       False
actor_1_name                  True
movie_title                  False
num_voted_users              False
cast_total_facebook_likes    False
actor_3_name                  True
facenumber_in_poster          True
plot_keywords                 True
movie_imdb_link              False
num_user_for_reviews          True
language                      True
country                       True
content_rating                True
budget                        True
title_year                    True
actor_2_facebook_likes        True
imdb_score                   False
aspect_ratio                  True
movie_facebook_likes         False
dtype: bool

In [22]:
#Voy ahora a tratar de extraer el id de IMDB de movie_imdb_link que, como no tiene nulos, podría ser la clave.
dfMovies5000['movie_imdb_link'].sample(5)

3556    http://www.imdb.com/title/tt0101507/?ref_=fn_t...
3543    http://www.imdb.com/title/tt0810900/?ref_=fn_t...
3881    http://www.imdb.com/title/tt5574490/?ref_=fn_t...
2529    http://www.imdb.com/title/tt0075909/?ref_=fn_t...
2739    http://www.imdb.com/title/tt0376968/?ref_=fn_t...
Name: movie_imdb_link, dtype: object

In [23]:
dfMovies5000['movie_imdb_link'].sample(5).str.split("/").map(lambda list: list[4])

4401    tt0363547
1582    tt1289401
3346    tt2017038
3568    tt0457530
4494    tt0179116
Name: movie_imdb_link, dtype: object

In [0]:
#Construyo una nueva columna con el identificador de imdb: id_imdb
dfMovies5000['imdb_id'] = dfMovies5000['movie_imdb_link'].str.split("/").map(lambda list: list[4])

In [25]:
dfMovies5000.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes', 'imdb_id'],
      dtype='object')

In [26]:
dfMovies5000[['imdb_id', 'movie_imdb_link']].sample(5)

,imdb_id,movie_imdb_link
4856,tt0113416,http://www.imdb.com/title/tt0113416/?ref_=fn_t...
3278,tt0072271,http://www.imdb.com/title/tt0072271/?ref_=fn_t...
1870,tt1175491,http://www.imdb.com/title/tt1175491/?ref_=fn_t...
4033,tt2106476,http://www.imdb.com/title/tt2106476/?ref_=fn_t...
382,tt0266987,http://www.imdb.com/title/tt0266987/?ref_=fn_t...


In [27]:
#Voy a asegurarme de que no tengo cadenas de longitud vacías en mi identificador. Así que, quitando blancos, me aseguro de que la longitud no es 0 y,
#en cambio, es siempre 9:
dfMovies5000['imdb_id'].str.strip().map(len).value_counts()

9    5043
Name: imdb_id, dtype: int64

In [28]:
#¿Tengo duplicados?
#(en dfMovies ya lo sé, por cómo descargué la info)
dfMovies5000['imdb_id'].duplicated().sum()
#Hay 124 claves que aparecen más de una vez (sin contar las duplicidades)

124

In [29]:
#Echo un vistazo a algunos:
dfMovies5000['imdb_id'][dfMovies5000['imdb_id'].duplicated()].head()
#Esto no me da los dos (o más) registros con igual clave imdb_id duplicada, sino sólo uno de ellos.

137    tt0918940
187    tt1673434
204    tt4262980
303    tt3332064
389    tt1502712
Name: imdb_id, dtype: object

In [30]:
dfMovies5000[dfMovies5000['imdb_id'] == 'tt0918940']

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,imdb_id
63,Color,David Yates,248.0,110.0,282.0,103.0,Alexander Skarsgård,11000.0,124051759.0,Action|Adventure|Drama|Romance,...,English,USA,PG-13,180000000.0,2016.0,10000.0,6.6,2.35,29000,tt0918940
137,Color,David Yates,248.0,110.0,282.0,103.0,Alexander Skarsgård,11000.0,124051759.0,Action|Adventure|Drama|Romance,...,English,USA,PG-13,180000000.0,2016.0,10000.0,6.6,2.35,29000,tt0918940


In [31]:
#Aunque hay 124 códigos imdb_id repetidos, sólo 45 registros registros están repetidos para todos los campos.
dfMovies5000.duplicated().sum()

45

In [32]:
#Y a nivel de imdb_id, gross y budget tengo 123 duplicados:
#REalmente, de momento, sólo voy a tomar la info de gross y budget (¿me podría interesar la de los likes???)
dfMovies5000[['imdb_id', 'gross', 'budget']].duplicated().sum()

123

In [0]:
#O sea: los duplicados a nivel de clave lo son también a nivel de gross y budget salvo por un registro. Voy a eliminar los
#duplicados a nivel de clave quedándome con el primero de los registros que me encuentre (SIMPLIFICACIÓN).
dfMovies5000 = dfMovies5000.drop_duplicates(subset='imdb_id', keep='first')

In [34]:
dfMovies5000.shape

(4919, 29)

In [35]:
#Y ahora ya puedo cruzar con dfMovies
#Voy a ver primero de cuántas pelis tengo budget y "gross", que son las variables que seguro voy a usar. No tengo claro que vaya a usar el resto.
dfMovies5000[['movie_imdb_link', 'gross', 'budget']].sample(5)

,movie_imdb_link,gross,budget
3681,http://www.imdb.com/title/tt0242508/?ref_=fn_t...,56007.0,5500000.0
953,http://www.imdb.com/title/tt1570728/?ref_=fn_t...,84244877.0,50000000.0
421,http://www.imdb.com/title/tt0233469/?ref_=fn_t...,40048332.0,85000000.0
3191,http://www.imdb.com/title/tt0107983/?ref_=fn_t...,3275585.0,11500000.0
3355,http://www.imdb.com/title/tt0110912/?ref_=fn_t...,107930000.0,8000000.0


In [36]:
#Ya vimos que gross y budget tienen nulos.
dfMovies5000[['movie_imdb_link', 'gross', 'budget']].isnull().any()

movie_imdb_link    False
gross               True
budget              True
dtype: bool

In [37]:
dfMovies5000[['movie_imdb_link', 'gross', 'budget']].count()

movie_imdb_link    4919
gross              4056
budget             4434
dtype: int64

In [0]:
#Cuando vaya a actualizar, estaría bien tener una marca del origen del dato. !!!!!!!!!!!!!!!!!!!!!!!

In [39]:
#Bueno, parece que aunque hay nulos no hay 0's.
dfMovies5000[(dfMovies5000['gross'].notnull()) & (dfMovies5000['gross']>0)].shape

(4056, 29)

In [40]:
#¿Y si exigo que los beneficios sean más de 50$? Bien: no cambia el número. Me da indicio de que el dato es más fiable que el de TMDB.
dfMovies5000[(dfMovies5000['gross'].notnull()) & (dfMovies5000['gross']>50)].shape

(4056, 29)

In [41]:
#Echo un vistazo de igual modo al budget: estupendo, no hay 0's
dfMovies5000[(dfMovies5000['budget'].notnull()) & (dfMovies5000['budget']>0)].shape

(4434, 29)

In [42]:
#Ni tampoco pelis con presupuesto menor o igual a 50$
dfMovies5000[(dfMovies5000['budget'].notnull()) & (dfMovies5000['budget']>50)].shape

(4434, 29)

In [43]:
#Tendré que homogeneizar criterios con dfMovies: no hay nulos pero sí 0's en revenue y budget.

#Otra cosa a tener en cuenta: no todas las pelis en dfMovies tenían identificador de imdb. Podría hacerme perder algunos registros en el cruce.
#Poco puedo hacer. No voy a anexar la info de lo que no cruce. Lo despreciaré.
dfMovies[['imdb_id', 'status']].count()

imdb_id    337380
status     393239
dtype: int64

In [44]:
dfMovies5000.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes', 'imdb_id'],
      dtype='object')

In [0]:
#Aquí no tengo el id de actor para cruzar con dfMovies. Así que voy a pasar de las variables "actor_1_facebook_likes", "actor_2_facebook_likes",
#"actor_3_facebook_likes", pero me voy a quedar con director_facebook_likes, movie_facebook_likes y "cast_total_facebook_likes". 
#Esta última, ¿suma las anteriores o es realmente la suma total de todo el cast?

In [46]:
#Esto no debería devolver registros:
dfMovies5000[dfMovies5000['actor_1_facebook_likes']+dfMovies5000['actor_2_facebook_likes']+dfMovies5000['actor_3_facebook_likes']>dfMovies5000['cast_total_facebook_likes']]
#Guay, no los devuelve.

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,imdb_id


In [47]:
#¿La suma total de likes de todo el reparto suele ser mayor que la suma de likes de los 3 primers actores? Sí, suele serlo!
dfMovies5000[dfMovies5000['actor_1_facebook_likes']+dfMovies5000['actor_2_facebook_likes']+dfMovies5000['actor_3_facebook_likes']<dfMovies5000['cast_total_facebook_likes']].shape

(4746, 29)

In [48]:
#¿De cuántos años tengo pelis? De un montón!!
dfMovies5000['title_year'].value_counts().sort_index()
#Que equivale a:
#dfMovies5000.groupby('title_year')['imdb_id'].count()

1916.0      1
1920.0      1
1925.0      1
1927.0      1
1929.0      2
1930.0      1
1932.0      1
1933.0      2
1934.0      1
1935.0      1
1936.0      2
1937.0      2
1938.0      2
1939.0      3
1940.0      5
1941.0      1
1942.0      2
1943.0      1
1944.0      1
1945.0      4
1946.0      3
1947.0      3
1948.0      3
1949.0      2
1950.0      1
1951.0      3
1952.0      4
1953.0      4
1954.0      4
1955.0      2
         ... 
1987.0     32
1988.0     30
1989.0     32
1990.0     29
1991.0     31
1992.0     34
1993.0     48
1994.0     54
1995.0     70
1996.0     98
1997.0    117
1998.0    132
1999.0    167
2000.0    169
2001.0    183
2002.0    204
2003.0    169
2004.0    207
2005.0    216
2006.0    235
2007.0    198
2008.0    223
2009.0    253
2010.0    225
2011.0    224
2012.0    214
2013.0    231
2014.0    243
2015.0    211
2016.0     98
Name: title_year, Length: 91, dtype: int64

In [49]:
#¿De cuántos países tengo pelis? Muy sesgado al cine en inglés (aunque tengo pelis de otros países)
dfMovies5000['country'].value_counts().sort_values(ascending=False)

USA                     3711
UK                       434
France                   154
Canada                   124
Germany                   94
Australia                 53
India                     34
Spain                     33
China                     28
Italy                     23
Japan                     22
Mexico                    17
Hong Kong                 17
South Korea               14
New Zealand               13
Ireland                   12
Denmark                   11
Russia                    11
Brazil                     8
Norway                     8
South Africa               8
Sweden                     6
Thailand                   5
Netherlands                5
Iran                       4
Romania                    4
Argentina                  4
Poland                     4
Israel                     4
Czech Republic             3
                        ... 
Greece                     2
Taiwan                     2
Panama                     1
Aruba         

In [50]:
dfMovies5000.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes', 'imdb_id'],
      dtype='object')

####CRUCE CON LA INFO DE TMDB:

In [51]:
#Voy a cruzar con dfMovies. 
#¿Tengo duplicados en el index? No me hace falta para el cruce pero debería haberlo visto ya:
dfMovies.index.duplicated().sum()

0

In [52]:
#Recuerdo que tengo nulos en dfMovies en imdb_id:
dfMovies[['imdb_id', 'status']].isnull().any()

imdb_id     True
status     False
dtype: bool

In [53]:
#¿Cuántos nulos?
dfMovies[['imdb_id', 'status']].isnull().sum()

imdb_id    55859
status         0
dtype: int64

In [54]:
#Resulta que además de nulos, hay valores ''
dfMovies[dfMovies['imdb_id']==''].shape[0]

45838

In [55]:
#Los veo aquí, donde además veo un valor que aparece dos veces: tt1564005
dfMovies['imdb_id'].value_counts()[0:5]

             45838
tt1564005        2
tt0210607        1
tt0415824        1
tt1000702        1
Name: imdb_id, dtype: int64

In [56]:
#Excluyendo los nulos y los '', parece que mi único duplicado es ese valor:
dfMovies[(dfMovies['imdb_id'].notnull()) & (dfMovies['imdb_id']!='')]['imdb_id'].duplicated().sum()

1

In [0]:
#Debería sustituir las cadenas de longitud vacía por nulos, para tener un criterio homogéneo. Lo hago:
#Podría mejorarlo (si necesito aplicarlo a otros) y convertir cualquier cadena de longitud vacía en nulos.
#Esta función se aplica a cualquier tipo de dato y devuelvo un None si la entrada es ''
def fill_blank_with_nones(value):
    if value=='':
        return None
    else:
        return value

In [0]:
dfMovies['imdb_id']=dfMovies['imdb_id'].map(fill_blank_with_nones)

In [59]:
#Veo si lo he hecho bien: ¿cuántos nulos tengo ahora? Perfecto!
dfMovies['imdb_id'].isnull().sum()

101697

In [60]:
#Voy a echar un vistazo al duplicado:
dfMovies[dfMovies['imdb_id']=='tt1564005']
#Realmente, se refiere a la misma peli. Aunque algún campo es diferente. Lo dejo estar de momento. Tampoco sé seguro que sea un error.

,belongs_to_collection_id,belongs_to_collection_name,budget,credits_cast,credits_crew,genres,id,imdb_id,keywords,original_language,...,reviews_total_pages,reviews_total_results,runtime,spoken_languages,status,tagline,title,titles,vote_average,vote_count
id,,,,,,,,,,,,,,,,,,,,,
101318,NaN,None,0,"[{'cast_id': 1004, 'character': 'Joss Jenner',...","[{'credit_id': '52fe49efc3a36847f81ae499', 'de...","[{'id': 18, 'name': 'Drama'}]",101318,tt1564005,[],en,...,0,0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Lies Between Friends,"[{'iso_3166_1': 'ES', 'title': 'Asesinato entr...",4.8,4
545015,NaN,None,0,"[{'cast_id': 0, 'character': 'Joss Jenner', 'c...",[],"[{'id': 9648, 'name': 'Mystery'}, {'id': 10770...",545015,tt1564005,[],en,...,0,0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Lies Between Friends,[],5.0,1


In [61]:
#Voy a cruzar. Antes controlo cuántos cruzan:
dfMovies.merge(dfMovies5000, how='inner', left_on='imdb_id', right_on='imdb_id').shape

(4798, 58)

In [62]:
#Estupendo: cruzan la mayoría.
dfMovies5000.shape

(4919, 29)

In [63]:
dfMovies.merge(dfMovies5000, how='inner', left_on='imdb_id', right_on='imdb_id').columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget_x',
       'credits_cast', 'credits_crew', 'genres_x', 'id', 'imdb_id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'release_dates', 'revenue', 'reviews_page', 'reviews_results',
       'reviews_total_pages', 'reviews_total_results', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'titles',
       'vote_average', 'vote_count', 'color', 'director_name',
       'num_critic_for_reviews', 'duration', 'director_facebook_likes',
       'actor_3_facebook_likes', 'actor_2_name', 'actor_1_facebook_likes',
       'gross', 'genres_y', 'actor_1_name', 'movie_title', 'num_voted_users',
       'cast_total_facebook_likes', 'actor_3_name', 'facenumber_in_poster',
       'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews', 'language',
       'country', 'content_rating', 'budget_y', 'title

In [64]:
dfMovies5000.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes', 'imdb_id'],
      dtype='object')

In [0]:
#Quiero cambiar el nombre de algunas columnas antes de cruzar, para que sean más fácilmente identificables:
def change_column_names(str):
    if str == 'budget':
        return 'imdb_budget'
    elif str == 'gross':
        return 'imdb_revenue'
    else:
        return str

In [0]:
dfMovies5000.columns = list(map(change_column_names, dfMovies5000.columns.tolist()))

In [67]:
dfMovies5000.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'imdb_revenue', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'imdb_budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes', 'imdb_id'],
      dtype='object')

In [0]:
#Hago el cruce:
dfMovies = dfMovies.merge(dfMovies5000[['imdb_id', 'imdb_revenue', 'imdb_budget', 'director_facebook_likes', 'cast_total_facebook_likes', 'movie_facebook_likes']], how='left', left_on='imdb_id', right_on='imdb_id')

In [69]:
dfMovies.shape

(393239, 35)

In [70]:
#Pero ahora tengo distinto criterio en budget y revenue para tmdb e imdb:
dfMovies[['budget', 'revenue', 'imdb_budget', 'imdb_revenue']].isnull().sum()

budget               0
revenue              0
imdb_budget     388825
imdb_revenue    389188
dtype: int64

In [71]:
dfMovies[['budget', 'revenue', 'imdb_budget', 'imdb_revenue']].notnull().sum()

budget          393239
revenue         393239
imdb_budget       4414
imdb_revenue      4051
dtype: int64

In [0]:
#Igualo criterios:
def change_none_to_zero(num):
    if num != num or num is None:
        return 0
    else:
        return num

In [0]:
lista = [None, 3, 'casa', np.NaN]

In [74]:
list(map(change_none_to_zero, lista))

[0, 3, 'casa', 0]

In [0]:
dfMovies['imdb_budget'] = dfMovies['imdb_budget'].map(change_none_to_zero)
dfMovies['imdb_revenue'] = dfMovies['imdb_revenue'].map(change_none_to_zero)

In [76]:
#Chequeo:
dfMovies[['budget', 'revenue', 'imdb_budget', 'imdb_revenue']].notnull().sum()

budget          393239
revenue         393239
imdb_budget     393239
imdb_revenue    393239
dtype: int64

In [77]:
#¿Cuántos 0s tengo?
dfMovies[['budget', 'revenue', 'imdb_budget', 'imdb_revenue']].applymap(lambda num: num==0).sum()

budget          374718
revenue         383224
imdb_budget     388825
imdb_revenue    389188
dtype: int64

In [78]:
#¿Cuántos valores distintos de 0 tengo?
dfMovies[['budget', 'revenue', 'imdb_budget', 'imdb_revenue']].applymap(lambda num: num!=0).sum()

budget          18521
revenue         10015
imdb_budget      4414
imdb_revenue     4051
dtype: int64

In [79]:
#Poniendo el foco en revenue, ¿en cuántos casos tengo dato distinto de 0 en imdb_revenue pero igual a 0 en revenue?
dfMovies[(dfMovies['revenue']==0) & (dfMovies['imdb_revenue']>0)].shape[0]

842

In [80]:
#¿Y en cuántos casos tengo dato distinto de 0 en imdb_revenue pero menor o igual que 50 en revenue? Recordar: imdb_revenue era siempre >50
dfMovies[(dfMovies['revenue']<=50) & (dfMovies['imdb_revenue']>0)].shape[0]

858

In [81]:
#Hago un ejercicio similar para budget:
dfMovies[(dfMovies['budget']==0) & (dfMovies['imdb_budget']>0)].shape[0]

680

In [82]:
dfMovies[(dfMovies['budget']<=50) & (dfMovies['imdb_budget']>0)].shape[0]

705

In [0]:
#Voy a adoptar un criterio simple ahora: tomo el dato de tmdb de revenue o budget, a no ser que sea 0 o <50, en cuyo caso tomo el de imdb.
#Afinarlo más adelante!!!!
dfMovies['revenue_final'] = dfMovies['revenue'] * ((dfMovies['revenue']>50) | ((dfMovies['revenue']<=50) & (dfMovies['imdb_revenue']==0))) + dfMovies['imdb_revenue'] * (~((dfMovies['revenue']>50) | ((dfMovies['revenue']<=50) & (dfMovies['imdb_revenue']==0))))
dfMovies['budget_final'] = dfMovies['budget'] * ((dfMovies['budget']>50) | ((dfMovies['budget']<=50) & (dfMovies['imdb_budget']==0))) + dfMovies['imdb_budget'] * (~((dfMovies['budget']>50) | ((dfMovies['budget']<=50) & (dfMovies['imdb_budget']==0))))

In [84]:
#Chequeo que lo he hecho bien:
dfMovies[(dfMovies['revenue']<=50) & (dfMovies['imdb_revenue']>0)][['revenue', 'imdb_revenue', 'revenue_final']].head()

,revenue,imdb_revenue,revenue_final
59,0,4542775.0,4542775.0
827,0,1276984.0,1276984.0
1344,0,20218.0,20218.0
1981,0,336456.0,336456.0
1992,0,6109075.0,6109075.0


In [85]:
dfMovies[(dfMovies['revenue']>50) & (dfMovies['imdb_revenue']>0)][['revenue', 'imdb_revenue', 'revenue_final']].head()

,revenue,imdb_revenue,revenue_final
136,399611,399611.0,399611.0
245,248384621,119793567.0,248384621.0
263,193772504,77679638.0,193772504.0
422,378858340,153629485.0,378858340.0
474,47536959,47536959.0,47536959.0


In [86]:
dfMovies[(dfMovies['revenue']<=50) & (dfMovies['revenue']>0) & (dfMovies['imdb_revenue']>0)][['revenue', 'imdb_revenue', 'revenue_final']].head()

,revenue,imdb_revenue,revenue_final
13636,8,8579684.0,8579684.0
38051,14,5204007.0,5204007.0
74130,12,11540112.0,11540112.0
79228,5,4771000.0,4771000.0
103735,32,25615792.0,25615792.0


In [87]:
#¿Cuántos valores distintos de 0 tengo?
dfMovies[['budget', 'revenue', 'imdb_budget', 'imdb_revenue', 'budget_final', 'revenue_final']].applymap(lambda num: num!=0).sum()

budget           18521
revenue          10015
imdb_budget       4414
imdb_revenue      4051
budget_final     19201
revenue_final    10857
dtype: int64

In [88]:
#Tengo un valor negativo!!!
dfMovies[['budget', 'revenue', 'imdb_budget', 'imdb_revenue', 'budget_final', 'revenue_final']].applymap(lambda num: num<0).sum()

budget           0
revenue          1
imdb_budget      0
imdb_revenue     0
budget_final     0
revenue_final    1
dtype: int64

####VOY A REDUCIR MI UNIVERSO A LOS REGISTROS PARA LOS QUE TENGO INGRESO

Si luego quiero hacer algún análisis del resto, siempre puedo volver a cargar los datos. Gano el que todo va a pesar mucho menos.

In [0]:
dfMovies = dfMovies[dfMovies['revenue_final']>0]

In [90]:
#Pierdo un registro!!! Porque tenía revenue<0!!
dfMovies.shape

(10856, 37)

In [93]:
#En algún momento me he cargado los índices:
dfMovies['id'].head()

id
141043    141043
19556      19556
376208    376208
59728      59728
48992      48992
Name: id, dtype: int64

In [0]:
#Así que los vuelvo a actualizar:
dfMovies.index = dfMovies["id"]
#Y así puedo aplicar las funciones de abajo, que están definidas para que el index sea el id de la pelícua, porque así van más rápidas:

In [94]:
#Genero estos sólo para poder generar un csv con los id's de cast and crew, para poder filtrarlos al importar a los jsons de people:
dfMoviesCast = columnProcessesMovies ('credits_cast', 'cast_id', 'character', 'gender', 'id', 'name', 'order')
print("dfMoviesCast generated")
dfMoviesCrew = columnProcessesMovies ('credits_crew', 'department', 'gender', 'id', 'job', 'name')
print("dfMoviesCrew generated")

dfMoviesCast generated
dfMoviesCrew generated


####GENERO UN CSV CON LOS ID'S DE CAST AND CREW CON REVENUE_FINAL >0 PARA REDUCIR LA IMPORTACIÓN DE FICHEROS JSON'S DE PEOPLE. IDEM CON LOS ID'S DE LAS PELÍCULAS CON LAS QUE VOY A TRABAJAR

In [95]:
#¿Cuántos ids tengo?
pd.DataFrame(pd.concat([dfMoviesCast['id'],dfMoviesCrew['id']]).unique(), columns=["id"]).shape

(203181, 1)

In [0]:
pd.DataFrame(pd.concat([dfMoviesCast['id'],dfMoviesCrew['id']]).unique(), columns=["id"]).to_csv('Cast_Crew_ids.csv')

In [97]:
pd.DataFrame(pd.concat([dfMoviesCast['id'],dfMoviesCrew['id']]).unique(), columns=["id"]).head()

,id
0,517
1,84497
2,17606
3,3051
4,4783


In [98]:
!head ./Cast_Crew_ids.csv

,id
0,517
1,84497
2,17606
3,3051
4,4783
5,10882
6,89822
7,1081875
8,43554


In [99]:
#Hago igual ahora con los id's de las películas:
dfMovies.index.duplicated().sum()

0

In [100]:
pd.DataFrame(dfMovies["id"].unique(), columns=["movie_id"]).head()

,movie_id
0,141043
1,19556
2,376208
3,59728
4,48992


In [0]:
pd.DataFrame(dfMovies["id"].unique(), columns=["movie_id"]).to_csv('Movies_ids.csv')

In [102]:
!head ./Movies_ids.csv

,movie_id
0,141043
1,19556
2,376208
3,59728
4,48992
5,169853
6,934
7,109451
8,399366


In [103]:
!ls -lh

total 3.2G
-rw-r--r-- 1 root root 2.5K Oct 28 10:31 adc.json
-rw-r--r-- 1 root root 2.7M Oct 28 20:40 Cast_Crew_ids.csv
drwx------ 3 root root 4.0K Oct 28 10:34 drive
-rw-r--r-- 1 root root 1.5M Oct 28 20:23 IMDB_5000_movie.csv
-rw-r--r-- 1 root root 117K Oct 28 20:42 Movies_ids.csv
-rw-r--r-- 1 root root 1.5G Oct 28 17:54 movies.zip
-rw-r--r-- 1 root root 1.8G Oct 28 16:36 people.zip
drwxr-xr-x 2 root root 4.0K Oct 25 17:24 sample_data
drwxr-xr-x 4 root root 4.0K Oct 28 11:33 themoviedb_data


In [0]:
!cp *_ids.csv /content/drive/"My Drive"

#### GESTIÓN DE LA MEMORIA

In [2]:
#Para saber el uso de memoria que estoy haciendo:
import os
import psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

134512640


In [115]:
#Se supone que esto libera la memoria, pero no veo efecto:
#import gc
#gc.collect()

8197

In [113]:
#Y así veo cuánta memoria ocupa este dataframe:
dfMovies.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10856 entries, 30 to 393212
Data columns (total 34 columns):
belongs_to_collection_id      1785 non-null float64
belongs_to_collection_name    1785 non-null object
budget                        10856 non-null int64
genres                        10856 non-null object
imdb_id                       10403 non-null object
keywords                      10856 non-null object
original_language             10856 non-null object
original_title                10856 non-null object
overview                      10856 non-null object
popularity                    10856 non-null float64
production_companies          10856 non-null object
production_countries          10856 non-null object
release_date                  10856 non-null object
release_dates                 10856 non-null object
revenue                       10856 non-null int64
reviews_page                  10856 non-null int64
reviews_results               10856 non-null object
reviews_

In [0]:
#Para eliminar de la memoria
#dfMovies5000Cleaned = pd.DataFrame()
#del dfMovies5000Cleaned